In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [6]:
from ktrain import text
import os

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Load dataset
file_path = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/merged/Familial_plus_batch_1_batch_2_merged.xlsx"
familial_df = pd.read_excel(file_path)


In [26]:
# Perform train-test split (keeping stratification)
train_familial, test_familial = train_test_split(
    familial_df, test_size=0.22, random_state=42, stratify=familial_df["label"]
)

# Verify counts
print(f"Familial Train: {len(train_familial)}, Test: {len(test_familial)}, Total: {len(familial_df)}")

# Count total samples
total_train = len(train_familial)
total_test = len(test_familial)

# Count positive (Class 1) samples
train_positives = train_familial["label"].sum()
test_positives = test_familial["label"].sum()

# Calculate percentages
train_positive_pct = (train_positives / total_train) * 100
test_positive_pct = (test_positives / total_test) * 100

# Print results
print(f"📊 Familial Dataset Train-Test Split Results:")
print(f"✅ Train: {total_train} samples (Positives: {train_positives}, {train_positive_pct:.2f}%)")
print(f"✅ Test: {total_test} samples (Positives: {test_positives}, {test_positive_pct:.2f}%)")
print(f"🔹 Total Dataset: {len(familial_df)} samples")


# Define save directory
save_dir = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/merged/fam_train_test"
os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

# Save train and test sets
train_familial.to_excel(f"{save_dir}/train_familial.xlsx", index=False)
test_familial.to_excel(f"{save_dir}/test_familial.xlsx", index=False)

print("✅ Familial train and test sets successfully created and saved!")

Familial Train: 1965, Test: 555, Total: 2520
📊 Familial Dataset Train-Test Split Results:
✅ Train: 1965 samples (Positives: 434, 22.09%)
✅ Test: 555 samples (Positives: 123, 22.16%)
🔹 Total Dataset: 2520 samples
✅ Familial train and test sets successfully created and saved!


In [10]:
MAXLEN = 150

In [11]:
X = train_familial['sentence']
y = train_familial['label']

In [12]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

In [13]:
model_name = 'distilbert-base-uncased'

In [14]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 20
	95percentile : 40
	99percentile : 57


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 42
	99percentile : 61


In [15]:
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)

In [16]:
distillbert_learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
262/262 [==============================] - 75s 237ms/step - loss: 0.3736 - accuracy: 0.8569 - val_loss: 0.3103 - val_accuracy: 0.8906
Epoch 2/5
262/262 [==============================] - 58s 220ms/step - loss: 0.2287 - accuracy: 0.9243 - val_loss: 0.3006 - val_accuracy: 0.8830
Epoch 3/5
262/262 [==============================] - 50s 190ms/step - loss: 0.1935 - accuracy: 0.9326 - val_loss: 0.2908 - val_accuracy: 0.8880
Epoch 4/5
262/262 [==============================] - 47s 178ms/step - loss: 0.1580 - accuracy: 0.9485 - val_loss: 0.2833 - val_accuracy: 0.8931
Epoch 5/5
262/262 [==============================] - 56s 213ms/step - loss: 0.1098 - accuracy: 0.9695 - val_loss: 0.3334 - val_accuracy: 0.8957


In [17]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

13/13 [==============================] - 6s 262ms/step
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       306
           1       0.77      0.76      0.76        87

    accuracy                           0.90       393
   macro avg       0.85      0.85      0.85       393
weighted avg       0.90      0.90      0.90       393



array([[286,  20],
       [ 21,  66]])

In [18]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

13/13 [==============================] - 3s 239ms/step
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       306
           1       0.77      0.76      0.76        87

    accuracy                           0.90       393
   macro avg       0.85      0.85      0.85       393
weighted avg       0.90      0.90      0.90       393



array([[286,  20],
       [ 21,  66]])

In [19]:
distillbert_learner.model.summary()

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [21]:
distillbert_test_data = test_familial['sentence'].tolist()
distillbert_test_label = test_familial['label'].tolist()

In [22]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [23]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [24]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 411, False Positive: 21, False Negative: 19, True Positive: 104


In [25]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.95       432
           1       0.83      0.85      0.84       123

    accuracy                           0.93       555
   macro avg       0.89      0.90      0.90       555
weighted avg       0.93      0.93      0.93       555
 



In [25]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.92      0.95       117
           1       0.72      0.92      0.81        25

    accuracy                           0.92       142
   macro avg       0.85      0.92      0.88       142
weighted avg       0.94      0.92      0.93       142
 



In [24]:
#distillbert_predictor.save('./model/distilbert_base_uncased_model') # 256 MB

In [38]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.8009703180009733
